In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
import math
import copy
import sklearn
import string
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import torchtext
from torchtext.data import get_tokenizer
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
uploaded = files.upload()

Saving dataset.csv to dataset.csv


In [5]:
#The emotions are classified into six categories: sadness (0), joy (1), love (2), anger (3), fear (4), and surprise (5).
data = pd.read_csv("/content/dataset.csv")

In [6]:
data.head()

,Unnamed: 0,text,label
0,0,i just feel really helpless and heavy hearted,4
1,1,ive enjoyed being able to slouch about relax a...,0
2,2,i gave up my internship with the dmrg and am f...,4
3,3,i dont know i feel so lost,0
4,4,i am a kindergarten teacher and i am thoroughl...,4


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 416809 entries, 0 to 416808
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  416809 non-null  int64 
 1   text        416809 non-null  object
 2   label       416809 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 9.5+ MB


In [8]:
data.isna().sum()

Unnamed: 0    0
text          0
label         0
dtype: int64

In [9]:
#The emotions are classified into six categories: sadness (0), joy (1), love (2), anger (3), fear (4), and surprise (5).
data['label'].value_counts()

label
1    141067
0    121187
3     57317
4     47712
2     34554
5     14972
Name: count, dtype: int64

In [10]:
stop_words = stopwords.words('english')
def clean_data(tweet) -> str:
  #Convert to Lower Case
  tweet = tweet.lower()

  #Remove Punctuation from the tweet
  tweet = tweet.translate(str.maketrans('', '', string.punctuation))

  #Remove stopwords from the tweet
  remove_stopwords = [word for word in tweet.split() if word not in stop_words]
  tweet = ' '.join(remove_stopwords)

  #Remove digits
  tweet = re.sub(r"\d",'',tweet)

  #remove extra space
  tweet = re.sub(r"\s+"," ",tweet)

  return tweet

In [11]:
data['model_input_text'] = data['text'].apply(clean_data)
data.head()

,Unnamed: 0,text,label,model_input_text
0,0,i just feel really helpless and heavy hearted,4,feel really helpless heavy hearted
1,1,ive enjoyed being able to slouch about relax a...,0,ive enjoyed able slouch relax unwind frankly n...
2,2,i gave up my internship with the dmrg and am f...,4,gave internship dmrg feeling distraught
3,3,i dont know i feel so lost,0,dont know feel lost
4,4,i am a kindergarten teacher and i am thoroughl...,4,kindergarten teacher thoroughly weary job take...


In [12]:
tokenizer = get_tokenizer("basic_english")
tokens = tokenizer("You can now install TorchText using pip!")
tokens

['you', 'can', 'now', 'install', 'torchtext', 'using', 'pip', '!']

In [13]:
def convert(lst):
  return ([i for i in lst.split()])

data['tokenized_sents'] = data['model_input_text'].apply(convert)
data.head()

,Unnamed: 0,text,label,model_input_text,tokenized_sents
0,0,i just feel really helpless and heavy hearted,4,feel really helpless heavy hearted,"[feel, really, helpless, heavy, hearted]"
1,1,ive enjoyed being able to slouch about relax a...,0,ive enjoyed able slouch relax unwind frankly n...,"[ive, enjoyed, able, slouch, relax, unwind, fr..."
2,2,i gave up my internship with the dmrg and am f...,4,gave internship dmrg feeling distraught,"[gave, internship, dmrg, feeling, distraught]"
3,3,i dont know i feel so lost,0,dont know feel lost,"[dont, know, feel, lost]"
4,4,i am a kindergarten teacher and i am thoroughl...,4,kindergarten teacher thoroughly weary job take...,"[kindergarten, teacher, thoroughly, weary, job..."


In [14]:
res = []
max_sent_len = 0
total_len = 0
#Follwoing line of codes are used to concatenate all input cleaned text into one sentence and define a lookup table with each word as a key and an integer value
#Input cleaned sentence with maximum length is also recorded. This value is used to find the length of input sequenced list after padding
for index, row in data.iterrows():
    res += row['tokenized_sents']
    sent_len = len(row['tokenized_sents'])
    total_len += sent_len
    if sent_len > max_sent_len:
      max_sent_len = sent_len
look_up = {s: i for i, s in enumerate(res)}


def convert_text(lst):
  """Converts list of words into list of integer sequences by mapping each word in look_up table defined above"""
  # res = list()
  # for word in lst:
  #   code = look_up[word]
  #   res.append(code)
  # return res
  return [look_up[word] for word in lst]

def padding(lst):
  """Pads list of integer sequences by adding zeroes in the front if the input sequence"""
  res = list()
  for i in range(max_sent_len - len(lst)):
    res.append(0)
  res += lst
  return res


data['tokenized_sequences'] = data['tokenized_sents'].apply(convert_text)
data['padded_sequences'] = data['tokenized_sequences'].apply(padding)
data.head()

,Unnamed: 0,text,label,model_input_text,tokenized_sents,tokenized_sequences,padded_sequences
0,0,i just feel really helpless and heavy hearted,4,feel really helpless heavy hearted,"[feel, really, helpless, heavy, hearted]","[3899824, 3899187, 3898073, 3897963, 3835981]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1,ive enjoyed being able to slouch about relax a...,0,ive enjoyed able slouch relax unwind frankly n...,"[ive, enjoyed, able, slouch, relax, unwind, fr...","[3899364, 3882812, 3899815, 1377156, 3898375, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,2,i gave up my internship with the dmrg and am f...,4,gave internship dmrg feeling distraught,"[gave, internship, dmrg, feeling, distraught]","[3899634, 3878954, 31, 3899780, 3896437]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,3,i dont know i feel so lost,0,dont know feel lost,"[dont, know, feel, lost]","[3899673, 3899722, 3899824, 3898864]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,4,i am a kindergarten teacher and i am thoroughl...,4,kindergarten teacher thoroughly weary job take...,"[kindergarten, teacher, thoroughly, weary, job...","[3884466, 3897100, 3897666, 3886858, 3899761, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [15]:
size_of_verification_data = 0.1
size_of_test_data = 0.2

from sklearn.model_selection import train_test_split

X = data['padded_sequences']
y = data['label']

X_model, X_verify, y_model, y_verify = train_test_split(X,y,test_size=size_of_verification_data,shuffle=True)
X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=size_of_test_data, shuffle=True)

In [16]:
print(f"Info of X_verify: {X_verify.info()} \n\n")
print(f"Info of X_train: {X_train.info()} \n\n")
print(f"Info of X_test: {X_test.info()} \n\n")

<class 'pandas.core.series.Series'>
Index: 41681 entries, 269335 to 397274
Series name: padded_sequences
Non-Null Count  Dtype 
--------------  ----- 
41681 non-null  object
dtypes: object(1)
memory usage: 651.3+ KB
Info of X_verify: None 


<class 'pandas.core.series.Series'>
Index: 300102 entries, 139731 to 328717
Series name: padded_sequences
Non-Null Count   Dtype 
--------------   ----- 
300102 non-null  object
dtypes: object(1)
memory usage: 4.6+ MB
Info of X_train: None 


<class 'pandas.core.series.Series'>
Index: 75026 entries, 110380 to 47389
Series name: padded_sequences
Non-Null Count  Dtype 
--------------  ----- 
75026 non-null  object
dtypes: object(1)
memory usage: 1.1+ MB
Info of X_test: None 




In [17]:
from torch.utils.data import DataLoader
BATCH_SIZE = 16


train_dataloader = DataLoader(list(zip(X_train,y_train)), shuffle=True, batch_size=BATCH_SIZE)
for X_batch, y_batch in train_dataloader:
    # print(X_batch, y_batch)
    break

test_dataloader = DataLoader(list(zip(X_test,y_test)), shuffle=True, batch_size=BATCH_SIZE)
for X_batch, y_batch in test_dataloader:
    # print(X_batch, y_batch)
    break

verify_dataloader = DataLoader(list(zip(X_verify,y_verify)), shuffle=True, batch_size=BATCH_SIZE)
for X_batch, y_batch in verify_dataloader:
    # print(X_batch, y_batch)
    break

In [18]:
print(f"Length of Train Dataloader is: {len(train_dataloader)}")
print(f"Length of Test Dataloader is: {len(test_dataloader)}")
print(f"Length of verify Dataloader is: {len(verify_dataloader)}")

Length of Train Dataloader is: 18757
Length of Test Dataloader is: 4690
Length of verify Dataloader is: 2606


In [19]:
train_feature_batch, train_labels_batch = next(iter(train_dataloader))
len(train_feature_batch), len(train_labels_batch)

(79, 16)

In [20]:
# class Input_Embedding(nn.Module):
#   def __init__(self,vocab_size:int,d_model:int):
#     super().__init__()
#     self.look_up_table = nn.Embedding(vocab_size,d_model)
#     self.d_model = d_model

#   def forward(self,x):
#     return self.look_up_table(x)*(math.sqrt(self.d_model))

In [21]:
# # vocab size
# vocab_size = total_len
# # embedding dimensions
# d_model = 3
# embed_obj = Input_Embedding(vocab_size,d_model)
# for num in data['padded_sequences'].iloc[0]:
#   lion = embed_obj(torch.Tensor(num).long())
#   break

In [22]:
# vocab size
vocab_size = total_len
# embedding dimensions
d_model = 3

# generate the embedding layer
embeddings = torch.rand(vocab_size, d_model) # matrix of size (total_len, 3)

#Method for embedding the input data
def input_embedding_layer(lst):
  return embeddings[lst]*(math.sqrt(d_model))

In [23]:
data['embedded'] = data['padded_sequences'].apply(input_embedding_layer)
data.head()

,Unnamed: 0,text,label,model_input_text,tokenized_sents,tokenized_sequences,padded_sequences,embedded
0,0,i just feel really helpless and heavy hearted,4,feel really helpless heavy hearted,"[feel, really, helpless, heavy, hearted]","[3899824, 3899187, 3898073, 3897963, 3835981]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[tensor(0.0830), tensor(1.6451), tensor(0.574..."
1,1,ive enjoyed being able to slouch about relax a...,0,ive enjoyed able slouch relax unwind frankly n...,"[ive, enjoyed, able, slouch, relax, unwind, fr...","[3899364, 3882812, 3899815, 1377156, 3898375, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[tensor(0.0830), tensor(1.6451), tensor(0.574..."
2,2,i gave up my internship with the dmrg and am f...,4,gave internship dmrg feeling distraught,"[gave, internship, dmrg, feeling, distraught]","[3899634, 3878954, 31, 3899780, 3896437]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[tensor(0.0830), tensor(1.6451), tensor(0.574..."
3,3,i dont know i feel so lost,0,dont know feel lost,"[dont, know, feel, lost]","[3899673, 3899722, 3899824, 3898864]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[tensor(0.0830), tensor(1.6451), tensor(0.574..."
4,4,i am a kindergarten teacher and i am thoroughl...,4,kindergarten teacher thoroughly weary job take...,"[kindergarten, teacher, thoroughly, weary, job...","[3884466, 3897100, 3897666, 3886858, 3899761, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[tensor(0.0830), tensor(1.6451), tensor(0.574..."


In [27]:
def pos_encoding(max_len,d_model,n):
  pe_matrix = np.zeros(max_len*d_model).reshape(max_len,d_model)

  for k in np.arange(max_len):
    for i in np.arange(d_model//2):
      theta = k / (n**((2*i)/d_model))
      pe_matrix[k,2*i] = math.sin(theta)
      pe_matrix[k,2*i+1] = math.cos(theta)
  return pe_matrix

In [34]:
pos_encodings = pos_encoding(max_sent_len,d_model,10000)
pos_encodings

array([[ 0.        ,  1.        ,  0.        ],
       [ 0.84147098,  0.54030231,  0.        ],
       [ 0.90929743, -0.41614684,  0.        ],
       [ 0.14112001, -0.9899925 ,  0.        ],
       [-0.7568025 , -0.65364362,  0.        ],
       [-0.95892427,  0.28366219,  0.        ],
       [-0.2794155 ,  0.96017029,  0.        ],
       [ 0.6569866 ,  0.75390225,  0.        ],
       [ 0.98935825, -0.14550003,  0.        ],
       [ 0.41211849, -0.91113026,  0.        ],
       [-0.54402111, -0.83907153,  0.        ],
       [-0.99999021,  0.0044257 ,  0.        ],
       [-0.53657292,  0.84385396,  0.        ],
       [ 0.42016704,  0.90744678,  0.        ],
       [ 0.99060736,  0.13673722,  0.        ],
       [ 0.65028784, -0.75968791,  0.        ],
       [-0.28790332, -0.95765948,  0.        ],
       [-0.96139749, -0.27516334,  0.        ],
       [-0.75098725,  0.66031671,  0.        ],
       [ 0.14987721,  0.98870462,  0.        ],
       [ 0.91294525,  0.40808206,  0.   

In [46]:
def add_pos_encoding(lst):
  new_lst = copy.deepcopy(lst)
  for i,item in enumerate(new_lst):
    item[0] += pos_encodings[i][0]
    item[1] += pos_encodings[i][1]
    item[2] += pos_encodings[i][2]
  return new_lst

data['pos_embedded'] = data['embedded'].apply(add_pos_encoding)
data.head()

,Unnamed: 0,text,label,model_input_text,tokenized_sents,tokenized_sequences,padded_sequences,embedded,pos_embedded
0,0,i just feel really helpless and heavy hearted,4,feel really helpless heavy hearted,"[feel, really, helpless, heavy, hearted]","[3899824, 3899187, 3898073, 3897963, 3835981]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[tensor(0.0830), tensor(3.6451), tensor(0.574...","[[tensor(0.0830), tensor(4.6451), tensor(0.574..."
1,1,ive enjoyed being able to slouch about relax a...,0,ive enjoyed able slouch relax unwind frankly n...,"[ive, enjoyed, able, slouch, relax, unwind, fr...","[3899364, 3882812, 3899815, 1377156, 3898375, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[tensor(0.0830), tensor(3.6451), tensor(0.574...","[[tensor(0.0830), tensor(4.6451), tensor(0.574..."
2,2,i gave up my internship with the dmrg and am f...,4,gave internship dmrg feeling distraught,"[gave, internship, dmrg, feeling, distraught]","[3899634, 3878954, 31, 3899780, 3896437]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[tensor(0.0830), tensor(3.6451), tensor(0.574...","[[tensor(0.0830), tensor(4.6451), tensor(0.574..."
3,3,i dont know i feel so lost,0,dont know feel lost,"[dont, know, feel, lost]","[3899673, 3899722, 3899824, 3898864]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[tensor(0.0830), tensor(3.6451), tensor(0.574...","[[tensor(0.0830), tensor(4.6451), tensor(0.574..."
4,4,i am a kindergarten teacher and i am thoroughl...,4,kindergarten teacher thoroughly weary job take...,"[kindergarten, teacher, thoroughly, weary, job...","[3884466, 3897100, 3897666, 3886858, 3899761, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[tensor(0.0830), tensor(3.6451), tensor(0.574...","[[tensor(0.0830), tensor(4.6451), tensor(0.574..."


In [52]:
data.to_csv("/content/dataset.csv", sep=',', index=False, encoding='utf-8')

In [53]:
data.head()

,Unnamed: 0,text,label,model_input_text,tokenized_sents,tokenized_sequences,padded_sequences,embedded,pos_embedded
0,0,i just feel really helpless and heavy hearted,4,feel really helpless heavy hearted,"[feel, really, helpless, heavy, hearted]","[3899824, 3899187, 3898073, 3897963, 3835981]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[tensor(0.0830), tensor(3.6451), tensor(0.574...","[[tensor(0.0830), tensor(4.6451), tensor(0.574..."
1,1,ive enjoyed being able to slouch about relax a...,0,ive enjoyed able slouch relax unwind frankly n...,"[ive, enjoyed, able, slouch, relax, unwind, fr...","[3899364, 3882812, 3899815, 1377156, 3898375, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[tensor(0.0830), tensor(3.6451), tensor(0.574...","[[tensor(0.0830), tensor(4.6451), tensor(0.574..."
2,2,i gave up my internship with the dmrg and am f...,4,gave internship dmrg feeling distraught,"[gave, internship, dmrg, feeling, distraught]","[3899634, 3878954, 31, 3899780, 3896437]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[tensor(0.0830), tensor(3.6451), tensor(0.574...","[[tensor(0.0830), tensor(4.6451), tensor(0.574..."
3,3,i dont know i feel so lost,0,dont know feel lost,"[dont, know, feel, lost]","[3899673, 3899722, 3899824, 3898864]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[tensor(0.0830), tensor(3.6451), tensor(0.574...","[[tensor(0.0830), tensor(4.6451), tensor(0.574..."
4,4,i am a kindergarten teacher and i am thoroughl...,4,kindergarten teacher thoroughly weary job take...,"[kindergarten, teacher, thoroughly, weary, job...","[3884466, 3897100, 3897666, 3886858, 3899761, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[tensor(0.0830), tensor(3.6451), tensor(0.574...","[[tensor(0.0830), tensor(4.6451), tensor(0.574..."


In [55]:
from google.colab import files

# Download the file
files.download('/content/dataset.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>